# Tarea 3: explorar y hacer consultas sobre los datos del kernel de PySpark

En este cuaderno, hará lo siguiente:

- Aprenderá a usar un cuaderno de Studio para explorar visualmente, autenticarse y conectarse a un clúster de Amazon EMR.
- Explorará y hará consultas sobre los datos con Spark ML. 
- Supervisará sus trabajos de Spark con la interfaz de usuario de Spark.

### Tarea 3.1: información sobre la sesión

Como está usando el kernel de PySpark, puede utilizar magic %%info de PySpark para mostrar información sobre la sesión actual.

In [ ]:
%%info

### Tarea 3.2: explorar y hacer consultas sobre los datos

Al usar el kernel de PySpark, se crean automáticamente un SparkContext y un HiveContext después de conectarse a un clúster de EMR. Puede usar HiveContext para hacer consultas sobre los datos en la tabla de Hive y hacer que estén disponibles en un DataFrame de Spark.

Use HiveContext para hacer consultas sobre Hive y observar las bases de datos y tablas.

In [ ]:
#query-hive

sqlContext = HiveContext(sqlContext)

dbs = sqlContext.sql("show databases")
dbs.show()

tables = sqlContext.sql("show tables")
tables.show()

Haga consultas sobre la tabla de datos Adult y lleve los datos a un DataFrame de Spark.

In [ ]:
#load-data

adult_df = sqlContext.sql("select * from adult_data").cache()

Use el DataFrame para revisar la forma y visualice las primeras cinco filas del conjunto de datos.

In [ ]:
#view-shape

print((adult_df.count(), len(adult_df.columns)))

#Show first 5 rows 
adult_df.head(5)

Para que el resultado sea más claro, convierta el DataFrame de Spark en un DataFrame de Pandas.

In [ ]:
#convert-dataframe

adult_df.limit(5).toPandas()

Algunos algoritmos de machine learning (ML), como la regresión lineal, requieren funciones numéricas. El conjunto de datos Adult que usa en este laboratorio incluye funciones categóricas, como **workclass**, **education**, **occupation**, **marital status**, **relationship**, **race** y **sex** (clase de trabajo, educación, ocupación, estado civil, relación, raza y sexo).

El siguiente bloque de código muestra cómo usar StringIndexer y OneHotEncoderEstimator para convertir variables categóricas en un conjunto de variables numéricas que asumen valores de 0 y 1.

- StringIndexer convierte una columna de valores de cadena en una columna de índices de etiquetas. 
- OneHotEncoderEstimator asigna una columna de índices de categorías a una columna de vectores binarios, con un máximo de un "1" en cada fila, que indica el índice de categoría de esa fila.

La codificación one-hot en Spark es un proceso de dos pasos. Primero, se usa **StringIndexer** y, luego, **OneHotEncoder**.

Consulte [StringIndexer](https://spark.apache.org/docs/latest/ml-features.html#stringindexer) y [OneHotEncoder](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder) para obtener más información.

In [ ]:
#convert-variables

from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator

categorical_variables = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

indexers = [StringIndexer(inputCol=column, outputCol=column+"-index") for column in categorical_variables]

encoder = OneHotEncoderEstimator(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=["{0}-encoded".format(indexer.getOutputCol()) for indexer in indexers]
)

La clase VectorAssembler toma varias columnas como entrada. Como resultado, arroja una sola columna que contiene una matriz de valores.

Consulte [VectorAssembler](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler) para obtener más información sobre este ensamblador.

In [ ]:
#vector-assembler

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="categorical-features"
)

Una canalización es una lista ordenada de transformadores y estimadores. Puede definir una canalización para automatizar y garantizar la repetición de las transformaciones que se aplicarán a un conjunto de datos. En este paso, debe definir la canalización y, luego, aplicarla al conjunto de datos.

Al igual que **StringIndexer**, una canalización es un **estimador**. El método pipeline.fit() muestra un **PipelineModel**, que es un **transformador**.

Consulte [Canalizaciones](https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline) para obtener más información sobre las canalizaciones de machine learning.

In [ ]:
#pyspark-pipelines

from pyspark.ml import Pipeline

# Define the pipeline based on the stages created in previous steps.
pipeline = Pipeline(stages=indexers + [encoder, assembler])

# Define the pipeline model.
pipelineModel = pipeline.fit(adult_df)

# Apply the pipeline model to the dataset.
adult_df = pipelineModel.transform(adult_df)

Revise todas las columnas diferentes que se crearon en el paso anterior.

In [ ]:
#print-schema

adult_df.printSchema()

Después de aplicar las transformaciones, una sola columna contendrá una matriz con todas las variables categóricas codificadas.

In [ ]:
#view-categorical-features

adult_df.select('categorical-features').show(truncate=False)

Ahora, codifique la etiqueta objetivo.

In [ ]:
#encode-target

indexer = StringIndexer(inputCol='income', outputCol='label')

adult_df = indexer.fit(adult_df).transform(adult_df)

### Tarea 3.3: supervisar y depurar con la interfaz de usuario de Spark

En esta sección, usará la interfaz de usuario de Spark para supervisar e inspeccionar el rendimiento de los trabajos de Spark que ejecutó en los pasos anteriores.

Obtenga información sobre la sesión actual de Spark.

In [ ]:
%%info

Puede encontrar hipervínculos para la **interfaz de usuario de Spark** y para el **registro del controlador**. En este laboratorio, el vínculo al **registro del controlador** está inactivo. La URL prefirmada de la **interfaz de usuario de Spark** se genera en el momento de la conexión al clúster de EMR. Si elige este vínculo, se lo dirigirá a la interfaz de usuario de Spark para inspeccionar las ejecuciones de trabajos de Spark en un navegador web. Estas métricas son útiles para ajustar el rendimiento. 

A continuación, se presentan algunas de las funciones importantes que debe buscar en el servidor de Spark:
- La pestaña **Jobs** (Trabajos) muestra el estado de todos los trabajos de Spark en esta aplicación de Spark.
- En la sección de resumen, la sección **Event Timeline** (Cronología del evento) muestra las diversas etapas de la ejecución.
- La sección **Completed Jobs** (Trabajos finalizados) se muestra en formato tabular. En la sección **Completed Jobs** (Trabajos finalizados), puede elegir un trabajo para revisar información sobre las etapas de las tareas incluidas en esta.
- Con **DAG Visualization** (Visualización de DAG), puede explorar las tareas que ejecutó antes. Al igual que con la vista de **Event Timeline** (Cronología del evento), con **DAG visualization** (Visualización de DAG), puede elegir una etapa y expandir los detalles incluidos en esta.

### Conclusión

¡Felicitaciones! Se conectó correctamente a un clúster de EMR y exploró e hizo consultas sobre los datos con PySpark.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, haga lo siguiente:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.